In [1]:
from coir.data_loader import get_tasks
from coir.evaluation import COIR
from coir.models import YourCustomDEModel
from datasets import load_dataset, Dataset, concatenate_datasets
import pandas as pd
from query_expansion import expand_queries_and_qrels
from functions import * 
from collections import defaultdict

/work/pi_wenlongzhao_umass_edu/27/vaishnavisha/envs/virtual-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


in tasks 
fetched data from hf
data loader init


20604it [00:00, 178416.91it/s]


In [2]:
cosqa_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/Cosqa/CoSQA_explanations_query_code.csv'
cosqa_deepseek_updated_path='/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/CS696DS-Oracle-Retrieving-Code-Explanations/Explanation_Generation/Cosqa/postprocessing/output/COSQA_deepseek_explanations_clean.csv'
csn_expl_path = '/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/explanations/CodeSearchNet_Python_deepseek_allSplits.csv'

#### duplicates

In [19]:
duplicate_code_groups = df.groupby('code').filter(lambda x: len(x) > 1)
num_duplicate_code_groups = duplicate_code_groups['code'].nunique()
print(f"Number of code groups with >1 item: {num_duplicate_code_groups} out of {len(df)}")

Number of code groups with >1 item: 3406 out of 20604


In [24]:
duplicate_code_df = df.groupby('code').filter(lambda x: len(x) > 1)

result = (
    duplicate_code_df
    .groupby('code')
    .agg({
        'corpus_id': lambda x: list(set(x)),
        'query_id': lambda x: list(set(x))
    })
    .reset_index()
    .rename(columns={'corpus_id': 'corpus_ids_with_same_code', 'query_id': 'query_ids_for_those_corpus'})
)

print(result.head())


                                                code  \
0  async def _send_plain_text(self, request: Requ...   
1  async def _thread_coro(self, *args):\n        ...   
2  async def async_input(prompt):\n    """\n    P...   
3  async def async_run(self) -> None:\n        ""...   
4  async def cursor(self) -> Cursor:\n        """...   

                           corpus_ids_with_same_code  \
0              [d3961, d1874, d20448, d6227, d10337]   
1  [d6144, d118, d5150, d12781, d7622, d1889, d82...   
2  [d2781, d16258, d16413, d4944, d9133, d9137, d...   
3           [d19465, d20020, d16870, d19314, d19313]   
4                   [d16659, d18387, d19386, d17979]   

                          query_ids_for_those_corpus  
0              [q20448, q1874, q3961, q6227, q10337]  
1  [q11702, q5150, q6147, q7622, q8299, q118, q18...  
2  [q4944, q16413, q9137, q16258, q20250, q9133, ...  
3           [q20020, q19314, q16870, q19465, q19313]  
4                   [q17979, q19386, q16659, q18387

In [17]:
duplicate_code_groups = df.groupby('cleaned_code').filter(lambda x: len(x) > 1)

num_duplicate_code_groups = duplicate_code_groups['cleaned_code'].nunique()

print(f"Number of cleaned_code groups with >1 item: {num_duplicate_code_groups} out of {len(df)}")

Number of cleaned_code groups with >1 item: 3406 out of 20604


In [16]:
duplicate_code_groups = df.groupby('explanation_deepseek_1_cleaned').filter(lambda x: len(x) > 1)

num_duplicate_code_groups = duplicate_code_groups['explanation_deepseek_1_cleaned'].nunique()

print(f"Number of explanation_deepseek_1_cleaned groups with >1 item: {num_duplicate_code_groups} out of {len(df)}")

Number of explanation_deepseek_1_cleaned groups with >1 item: 93 out of 20604


#### Query Expansion

In [ ]:
cosqa_deepseek_qe_path = '/work/pi_wenlongzhao_umass_edu/27/janet/query_expansion/results/cosqa_queries_expanded_deepseek_temp_0.csv'
cosqa_deepseek_qe_df = pd.read_csv(cosqa_deepseek_qe_path)
dataset_name = "cosqa"
tasks = load_data(dataset_name)
corpus, queries, qrels = tasks[dataset_name]
new_queries, new_qrels = expand_queries_and_qrels(cosqa_deepseek_qe_df, qrels)
tasks[dataset_name] = (corpus, new_queries, new_qrels)

In [ ]:
type(new_queries)

In [ ]:
for qid, query in new_queries.items():
    if isinstance(query, float) and pd.isna(query):
        print(qid, 'nan')
        continue

#### Running the pipeline

In [3]:
dataset_name = "cosqa"
tasks = load_data(dataset_name)
corpus, queries, qrels = tasks[dataset_name]

in tasks 
fetched data from hf
data loader init


20604it [00:00, 187510.45it/s]


In [7]:
df= pd.read_csv(cosqa_deepseek_updated_path)
corpus = update_corpus_with_cleaned_code(corpus, df) # updating the corpus with no comments in code

In [8]:
corpus['d1']

{'text': '\ndef writeBoolean(self, n):\n\n    t = TYPE_BOOL_TRUE\n\n    if n is False:\n        t = TYPE_BOOL_FALSE\n\n    self.stream.write(t)',
 'title': ''}

In [9]:
queries['q20105']

'sort by a token in string python'

In [10]:
qrels['q20105']

{'d20105': 1}

In [15]:
add_expl(tasks, "cosqa", cosqa_deepseek_updated_path, "explanation_deepseek_1_cleaned") # adding explanation instead of code

{'text': 'The code writes a boolean value to a stream. It checks if the input is False, and if so, assigns the corresponding boolean value (False) to the stream. Otherwise, it assigns the True value.The code checks if the input `n` is `False`. If it is, the code assigns `False` to the stream. Otherwise, it assigns `True` to the stream.', 'title': ''}
Total docs in corpus after replacement: 20604
Number of docs still missing 'text': 0


In [16]:
corpus, queries, qrels = tasks[dataset_name]

In [17]:
corpus['d1']

{'text': 'The code writes a boolean value to a stream. It checks if the input is False, and if so, assigns the corresponding boolean value (False) to the stream. Otherwise, it assigns the True value.The code checks if the input `n` is `False`. If it is, the code assigns `False` to the stream. Otherwise, it assigns `True` to the stream.',
 'title': ''}

In [18]:
run("BAAI/bge-base-en", tasks, "deepseek1", "dres", "cosqa_upd")

YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

in exact_search.py
len of queries:  500


Encoding batches: 100%|██████████| 2/2 [00:00<00:00, 12.69batch/s]


len of corpus:  20604


Encoding batches: 100%|██████████| 81/81 [00:41<00:00,  1.95batch/s]


in evaluation.py: evaluating

in beir/retrieval/evaluation.py: evaluating now

Retrieval evaluation results saved to results/cosqa_upd/deepseek1/dres/BAAI/bge-base-en
{'cosqa': {'NDCG': {'NDCG@1': 0.292, 'NDCG@3': 0.30832, 'NDCG@5': 0.30761, 'NDCG@10': 0.32928, 'NDCG@100': 0.45347, 'NDCG@1000': 0.50864}, 'MAP': {'MAP@1': 0.08037, 'MAP@3': 0.14899, 'MAP@5': 0.18935, 'MAP@10': 0.24044, 'MAP@100': 0.3004, 'MAP@1000': 0.30548}, 'Recall': {'Recall@1': 0.08037, 'Recall@3': 0.1774, 'Recall@5': 0.24339, 'Recall@10': 0.35584, 'Recall@100': 0.70552, 'Recall@1000': 0.94313}, 'Precision': {'P@1': 0.292, 'P@3': 0.27533, 'P@5': 0.2472, 'P@10': 0.2046, 'P@100': 0.0466, 'P@1000': 0.00636}}}


In [14]:
run("BAAI/bge-base-en", tasks, "baseline", "dres", "cosqa_upd")

YourCustomDEModel init
COIR init!
in evaluation.py: loading up dres

in evaluation.py: retrieving

in beir/retrieval/evaluation.py: loading up search

in exact_search.py
len of queries:  500


Encoding batches: 100%|██████████| 2/2 [00:00<00:00,  4.34batch/s]


len of corpus:  20604


Encoding batches: 100%|██████████| 81/81 [00:26<00:00,  3.03batch/s]


in evaluation.py: evaluating

in beir/retrieval/evaluation.py: evaluating now

Retrieval evaluation results saved to results/cosqa_upd/baseline/dres/BAAI/bge-base-en
{'cosqa': {'NDCG': {'NDCG@1': 0.326, 'NDCG@3': 0.35481, 'NDCG@5': 0.3773, 'NDCG@10': 0.42404, 'NDCG@100': 0.5592, 'NDCG@1000': 0.59704}, 'MAP': {'MAP@1': 0.11474, 'MAP@3': 0.21285, 'MAP@5': 0.28507, 'MAP@10': 0.38257, 'MAP@100': 0.45915, 'MAP@1000': 0.46239}, 'Recall': {'Recall@1': 0.11474, 'Recall@3': 0.22666, 'Recall@5': 0.31962, 'Recall@10': 0.47724, 'Recall@100': 0.82137, 'Recall@1000': 0.98}, 'Precision': {'P@1': 0.326, 'P@3': 0.30733, 'P@5': 0.3004, 'P@10': 0.2628, 'P@100': 0.05444, 'P@1000': 0.00662}}}
